# Tree-based classification model

### Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier as RFC 
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *
import random
import pickle

### Read the dataset

In [2]:
df = pd.read_csv('Dataset/rgb_img_dataset_test_colorfulness.csv')

In [3]:
df = df.drop('filename', axis=1)

### Dataset example

In [4]:
df.head()

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,...,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,colorfulness,label
0,16-31,16-31,16-31,0.303225,0-15,0-15,0-15,0.235109,0-15,0-15,...,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0,35.539986,0
1,16-31,16-31,16-31,0.176878,240-255,240-255,240-255,0.091829,224-239,224-239,...,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0,13.672892,0
2,224-239,224-239,224-239,0.092203,208-223,208-223,208-223,0.082258,224-239,224-239,...,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0,9.019223,0
3,48-63,32-47,32-47,0.063760,112-127,32-47,0-15,0.059953,128-143,48-63,...,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0,64.195108,0
4,0-15,0-15,0-15,0.127613,192-207,176-191,160-175,0.112392,192-207,176-191,...,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0,25.580210,0


### Convert the format of each string range-column to integer order

In [5]:
# Convert the format of each string range-column to integer order
def convert_range_to_order(val):
    result = int(val.split('-')[0])/16
    return int(result)

In [6]:
for col_name in df.columns:
    if('_range_' in col_name):
        df[col_name] = df[col_name].apply(lambda x: convert_range_to_order(x))

### Result from convert_range_to_order() function

In [7]:
df.head()

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,...,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,colorfulness,label
0,1,1,1,0.303225,0,0,0,0.235109,0,0,...,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0,35.539986,0
1,1,1,1,0.176878,15,15,15,0.091829,14,14,...,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0,13.672892,0
2,14,14,14,0.092203,13,13,13,0.082258,14,14,...,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0,9.019223,0
3,3,2,2,0.063760,7,2,0,0.059953,8,3,...,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0,64.195108,0
4,0,0,0,0.127613,12,11,10,0.112392,12,11,...,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0,25.580210,0


In [8]:
df['label'].value_counts()

0    455
1    152
Name: label, dtype: int64

### Random split the dataframe into 2 sub-dataframes for training and testing

In [9]:
training_rate = 0.75
testing_rate = 1-training_rate
df_len = len(df)

In [10]:
all_set = set(range(0, df_len))
training_set = set(random.sample(range(0, df_len), int(training_rate*df_len)))
testing_set = all_set-training_set

In [11]:
training_idx = np.array(list(training_set))
testing_idx = np.array(list(testing_set))
training_idx.sort()
testing_idx.sort()

In [12]:
df_train = df.iloc[training_idx]
df_test = df.iloc[testing_idx]

### Functions

In [13]:
# model training process 
# input: image infomation (dataframe), training_ratio (float), set of model parameters (dictionary)
# output: model, score of model (4 floats)

def model_training(df, training_size, param): 
    # training_size is ratio of training data for train-test split (float)
    # param is set of parameter for model training (dict)
    
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = tts(X, y, train_size=training_size, random_state=42)
    
    sm = SMOTE(sampling_strategy = 1, random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train.ravel())
    
    if(param['model']=='GBC'):
        clf = GBC(max_depth=param['max_depth'], 
                  criterion=param['criterion'],
                  min_impurity_decrease=param['min_impurity_decrease'],
                  min_samples_leaf=param['min_samples_leaf'], random_state=42).fit(X_res, y_res)
    elif(param['model']=='RFC'):
        clf = RFC(max_depth=param['max_depth'], 
                  criterion=param['criterion'],
                  min_impurity_decrease=param['min_impurity_decrease'],
                  min_samples_leaf=param['min_samples_leaf'], random_state=42).fit(X_res, y_res)
    # clf = GBC(set_of_parameter)
    y_pred = clf.predict(X_test)
    
#     print('Confusion Matrix')
#     print(confusion_matrix(y_test, y_pred))
#     print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
#     print('Precision: {:.4f}'.format(precision_score(y_test, y_pred)))
#     print('Recall: {:.4f}'.format(recall_score(y_test, y_pred)))
    
    return clf, accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred) 

In [14]:
# random parameter process for parameter finding with genetic algorithm
# input: parameter name (string), model name (string)
# output: parameter value

def random_parameter(param_name, model_name=None):
    
    if(param_name=='model'):
        tmp = random.randint(1, 2)
        if(tmp==1):
            return 'GBC'
        elif(tmp==2):
            return 'RFC'
        
    if(param_name=='max_depth'):
        return random.randint(1, 10)
    
    if(param_name=='criterion'):
        if(model_name=='GBC'):
            tmp = random.randint(1, 2)
            if(tmp==1):
                return 'friedman_mse'
            elif(tmp==2):
                return 'mae'
        elif(model_name=='RFC'):
            tmp = random.randint(1, 2)
            if(tmp==1):
                return 'gini'
            elif(tmp==2):
                return 'entropy'
            
    if(param_name=='min_impurity_decrease'):
        return random.random()/10
    
    if(param_name=='min_samples_leaf'):
        if(random.random()<0.5):
            return random.random()/10
        else:
            return random.randint(1, 20)

In [15]:
# generate the set of parameter for a chromosome
# input: -
# output: set of parameter (dictionary)

def gen_param():
    model_name = random_parameter('model')
    param = {'model': model_name,
             'max_depth': random_parameter('max_depth'),
             'criterion': random_parameter('criterion', model_name),
             'min_impurity_decrease': random_parameter('min_impurity_decrease'),
             'min_samples_leaf': random_parameter('min_samples_leaf')}
    return param

In [16]:
# generate the set of parameter for a chromosome with mutation process
# input: set of original parameter (dictionary)
# output: set of modified parameter (dictionary)

def gen_param_crossover(param):
    rand_threshold = 0.3
    
    if(random.random()<rand_threshold):
        model_name = random_parameter('model')
    else:
        model_name = param['model']
        
    if(random.random()<rand_threshold):
        max_depth = random_parameter('max_depth')
    else:
        max_depth = param['max_depth']
    
    if(random.random()<rand_threshold):
        criterion = random_parameter('criterion')
    else:
        criterion = param['criterion']
        
    if(random.random()<rand_threshold):
        min_impurity_decrease = random_parameter('min_impurity_decrease')
    else:
        min_impurity_decrease = param['min_impurity_decrease']
        
    if(random.random()<rand_threshold):
        min_samples_leaf = random_parameter('min_samples_leaf')
    else:
        min_samples_leaf = param['min_samples_leaf']
        
    param = {'model': model_name,
             'max_depth': max_depth,
             'criterion': criterion,
             'min_impurity_decrease': min_impurity_decrease,
             'min_samples_leaf': min_samples_leaf}
    
    return param

In [17]:
# 1st generation of genetic algorithm (without crossover process)
# input: (if exists) best set of parameter, model, model evaluation
# output: list of set of parameters, list of models, list of model evaluation

def gen_1st(param_prev=None, model_prev=None, eval_prev=None):

    n_chro = 100
    
    if((param_prev is None)&(model_prev is None)):
        params = np.array([None])
        models = np.array([None for i in range(n_chro+1)])
        eval_val = np.array([0, 0, 0, 0], ndmin=2)
        count = 0
#     else:
#         acc_score, prec_score, rec_score, f1_score = evaluate(model_prev, x_test, y_test)
#         eval_val = np.array([acc_score, prec_score, rec_score, f1_score], ndmin=2)
#         params = param_prev.copy()
#         models = np.array([model_prev])
#         count = 1

    while(count<n_chro):
        try:
            param = gen_param()
            print('param: {}'.format(param))
            model, acc_score, prec_score, rec_score, f1_score = model_training(df_train, 0.75, param)
            eval_val = np.append(eval_val, np.array([acc_score, prec_score, rec_score, f1_score], ndmin=2), axis=0)
            params = np.append(params, [param])
            # models = np.append(models, [model])
            models[count+1] = model
            count = count + 1

        except:
            continue

    return params, models, eval_val


In [18]:
# 1st generation of genetic algorithm (without crossover process)
# input: best set of parameter, model, model evaluation from previous generation
# output: list of set of parameters, list of models, list of model evaluation

def gen_nth(param_best, model_best, eval_val_best):
    
    n_chro = 100
    
    # params = np.array([param_best])
    # models = np.array([model_best])
    models = np.array([None for i in range(n_chro)])
    params = param_best.copy()
    for i in range(len(model_best)):
        models[i] = model_best[i]
    # models = model_best.copy()
    eval_val = np.array(eval_val_best.copy(), ndmin=2)
    count = len(model_best)
    for i in range(len(model_best)):
        try:
            param = gen_param()
            print('param: {}'.format(param))
            model, acc_score, prec_score, rec_score, f1_score = model_training(df_train, 0.75, param)
            eval_val = np.append(eval_val, np.array([acc_score, prec_score, rec_score, f1_score], ndmin=2), axis=0)
            params = np.append(params, [param])
            # models = np.append(models, [model])
            models[count] = model
            count = count + 1
        except:
            continue


    while(count<n_chro):
        try:
            param = gen_param()
            print('param: {}'.format(param))
            model, acc_score, prec_score, rec_score, f1_score = model_training(df_train, 0.75, param)
            eval_val = np.append(eval_val, np.array([acc_score, prec_score, rec_score, f1_score], ndmin=2), axis=0)
            params = np.append(params, [param])
            # models = np.append(models, [model])
            models[count] = model
            count = count + 1
        except:
            continue

    return params, models, eval_val

In [19]:
# Save the set of parameters and model evalution
# input: generation (integer), list of set of parameters (list of dictionaries), list of model evaluations (list of arrays)
# output: -

def create_report(gen, params, eval_val):
    f = open('checkpoints/report_param_gen{:02d}.txt'.format(gen), 'w')
    st_idx = 0
    if(gen==1):
        st_idx = 1
    for i in range(st_idx, params.shape[0]):
        f.writelines('{}\n'.format(params[i]))
    f.close()
    f = open('checkpoints/report_eval_gen{:02d}.txt'.format(gen), 'w')
    for i in range(st_idx, eval_val.shape[0]):
        f.writelines('{}\n'.format(eval_val[i]))
    f.close()

In [20]:
# Main function (start without previous model)
# input: number of generations (integer), number of best chromosomes be keeping to next generations (integer)
# output: list of set of parameters, list of models, list of model evaluations in first 3 and the last generation.

def start_from_new(gen=15, n_select_max=3):

    print('GEN 1:')
    params, models, eval_val = gen_1st()
    create_report(1, params, eval_val)

    print('\n\n\nGEN 2:')
    tmp_params = params.copy()
    tmp_models = models.copy()
    tmp_eval_val = eval_val.copy()
    
    param_best_set = tmp_params[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
    model_best_set = tmp_models[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
    eval_val_best_set = tmp_eval_val[np.argsort(tmp_eval_val[:, 3])[::-1]][0:n_select_max]
    # model_best_set = list(set(model_best_set))
    # eval_val_best_set = list(set(eval_val_best_set))
    params_2nd, models_2nd, eval_val_2nd = gen_nth(param_best=param_best_set, model_best=model_best_set, eval_val_best=eval_val_best_set)
    create_report(2, params_2nd, eval_val_2nd)
    
    print('\n\n\nGEN 3:')
    tmp_params = params_2nd.copy()
    tmp_models = models_2nd.copy()
    tmp_eval_val = eval_val_2nd.copy()
    param_best_set = tmp_params[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
    model_best_set = tmp_models[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
    eval_val_best_set = tmp_eval_val[np.argsort(tmp_eval_val[:, 3])[::-1]][0:n_select_max]
    # model_best_set = list(set(model_best_set))
    # eval_val_best_set = list(set(eval_val_best_set))
    params_3rd, models_3rd, eval_val_3rd = gen_nth(param_best=param_best_set, model_best=model_best_set, eval_val_best=eval_val_best_set)
    create_report(3, params_3rd, eval_val_3rd)
    
    params_nth, models_nth, eval_val_nth = params_3rd.copy(), models_3rd.copy(), eval_val_3rd.copy()
    for i in range(gen-3):
        print('\n\n\nGEN {}:'.format(i+4))
        tmp_params = params_nth.copy()
        tmp_models = models_nth.copy()
        tmp_eval_val = eval_val_nth.copy()
        param_best_set = tmp_params[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
        model_best_set = tmp_models[list(np.argsort(tmp_eval_val[:, 3])[::-1])][0:n_select_max]
        eval_val_best_set = tmp_eval_val[np.argsort(tmp_eval_val[:, 3])[::-1]][0:n_select_max]
        # model_best_set = list(set(model_best_set))
        # eval_val_best_set = list(set(eval_val_best_set))
        params_nth, models_nth, eval_val_nth = gen_nth(param_best=param_best_set, model_best=model_best_set, eval_val_best=eval_val_best_set)
        create_report(i+4, params_nth, eval_val_nth)
        
    return params, models, eval_val, params_2nd, models_2nd, eval_val_2nd, params_3rd, models_3rd, eval_val_3rd, params_nth, models_nth, eval_val_nth

### Run

In [21]:
params, models, eval_val, params_2nd, models_2nd, eval_val_2nd, params_3rd, models_3rd, eval_val_3rd, params_nth, models_nth, eval_val_nth = start_from_new(gen=20, n_select_max=10)

GEN 1:
param: {'model': 'GBC', 'max_depth': 10, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.015433056947846746, 'min_samples_leaf': 6}
param: {'model': 'GBC', 'max_depth': 6, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.0532351059305781, 'min_samples_leaf': 0.0671876610556841}
param: {'model': 'GBC', 'max_depth': 7, 'criterion': 'mae', 'min_impurity_decrease': 0.015236885544546497, 'min_samples_leaf': 0.047382477445048435}


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1118: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "


param: {'model': 'RFC', 'max_depth': 8, 'criterion': 'entropy', 'min_impurity_decrease': 0.06585243202941282, 'min_samples_leaf': 0.09342866281589692}
param: {'model': 'GBC', 'max_depth': 3, 'criterion': 'mae', 'min_impurity_decrease': 0.05233496848248694, 'min_samples_leaf': 0.02856307970814098}


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1118: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "


param: {'model': 'GBC', 'max_depth': 4, 'criterion': 'mae', 'min_impurity_decrease': 0.022221862399948757, 'min_samples_leaf': 0.029175812622436348}


...

param: {'model': 'GBC', 'max_depth': 6, 'criterion': 'mae', 'min_impurity_decrease': 0.0851443124254907, 'min_samples_leaf': 0.09005537055530752}


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1118: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "


param: {'model': 'GBC', 'max_depth': 4, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.07724373852669028, 'min_samples_leaf': 2}
param: {'model': 'RFC', 'max_depth': 3, 'criterion': 'gini', 'min_impurity_decrease': 0.026437209140434858, 'min_samples_leaf': 12}
param: {'model': 'GBC', 'max_depth': 5, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.06970274079792224, 'min_samples_leaf': 16}
param: {'model': 'RFC', 'max_depth': 6, 'criterion': 'gini', 'min_impurity_decrease': 0.07624645961430665, 'min_samples_leaf': 16}
param: {'model': 'RFC', 'max_depth': 1, 'criterion': 'entropy', 'min_impurity_decrease': 0.06134544630942264, 'min_samples_leaf': 1}
param: {'model': 'GBC', 'max_depth': 4, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.00458417261436066, 'min_samples_leaf': 0.04403981877627565}
param: {'model': 'RFC', 'max_depth': 4, 'criterion': 'gini', 'min_impurity_decrease': 0.039578909956194785, 'min_samples_leaf': 0.07138982495718058}
param: {'model': 'RFC'

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1118: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "


param: {'model': 'RFC', 'max_depth': 4, 'criterion': 'entropy', 'min_impurity_decrease': 0.031046391431896515, 'min_samples_leaf': 8}
param: {'model': 'GBC', 'max_depth': 5, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.061018657421711406, 'min_samples_leaf': 0.0184254829570062}
param: {'model': 'GBC', 'max_depth': 5, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.044820431523457514, 'min_samples_leaf': 18}
param: {'model': 'GBC', 'max_depth': 10, 'criterion': 'friedman_mse', 'min_impurity_decrease': 0.033128710619956724, 'min_samples_leaf': 0.051087114450934404}


### Results

#### 1st gen

In [22]:
params[np.argsort(eval_val[:, 3])[::-1]]

array([{'model': 'RFC', 'max_depth': 2, 'criterion': 'entropy', 'min_impurity_decrease': 0.026831972084402512, 'min_samples_leaf': 3},
       {'model': 'RFC', 'max_depth': 2, 'criterion': 'entropy', 'min_impurity_decrease': 0.014011887389787392, 'min_samples_leaf': 0.013480633167493062},
       {'model': 'RFC', 'max_depth': 7, 'criterion': 'gini', 'min_impurity_decrease': 0.007989889719185194, 'min_samples_leaf': 5},
       {'model': 'RFC', 'max_depth': 3, 'criterion': 'entropy', 'min_impurity_decrease': 0.03253643133366811, 'min_samples_leaf': 7},
       {'model': 'RFC', 'max_depth': 10, 'criterion': 'gini', 'min_impurity_decrease': 0.031933292587132525, 'min_samples_leaf': 0.039355626319636496},
       {'model': 'RFC', 'max_depth': 6, 'criterion': 'gini', 'min_impurity_decrease': 0.028921224110944443, 'min_samples_leaf': 0.04590186336618278},
       {'model': 'RFC', 'max_depth': 10, 'criterion': 'entropy', 'min_impurity_decrease': 0.04917205243528113, 'min_samples_leaf': 0.0321378876

In [23]:
models[np.argsort(eval_val[:, 3])[::-1]]

array([RandomForestClassifier(criterion='entropy', max_depth=2,
                              min_impurity_decrease=0.026831972084402512,
                              min_samples_leaf=3, random_state=42)       ,
       RandomForestClassifier(criterion='entropy', max_depth=2,
                              min_impurity_decrease=0.014011887389787392,
                              min_samples_leaf=0.013480633167493062, random_state=42),
       RandomForestClassifier(max_depth=7, min_impurity_decrease=0.007989889719185194,
                              min_samples_leaf=5, random_state=42)                    ,
       RandomForestClassifier(criterion='entropy', max_depth=3,
                              min_impurity_decrease=0.03253643133366811,
                              min_samples_leaf=7, random_state=42)      ,
       RandomForestClassifier(max_depth=10, min_impurity_decrease=0.031933292587132525,
                              min_samples_leaf=0.039355626319636496, random_state=42)  ,

In [24]:
eval_val[np.argsort(eval_val[:, 3])[::-1]]

array([[0.78947368, 0.51282051, 0.8       , 0.625     ],
       [0.78947368, 0.51282051, 0.8       , 0.625     ],
       [0.79824561, 0.52777778, 0.76      , 0.62295082],
       [0.80701754, 0.54545455, 0.72      , 0.62068966],
       [0.78070175, 0.5       , 0.8       , 0.61538462],
       [0.78070175, 0.5       , 0.8       , 0.61538462],
       [0.78070175, 0.5       , 0.76      , 0.6031746 ],
       [0.78947368, 0.51428571, 0.72      , 0.6       ],
       [0.78947368, 0.51428571, 0.72      , 0.6       ],
       [0.77192982, 0.48717949, 0.76      , 0.59375   ],
       [0.78070175, 0.5       , 0.72      , 0.59016393],
       [0.76315789, 0.475     , 0.76      , 0.58461538],
       [0.76315789, 0.475     , 0.76      , 0.58461538],
       [0.75438596, 0.46341463, 0.76      , 0.57575758],
       [0.75438596, 0.46341463, 0.76      , 0.57575758],
       [0.75438596, 0.46341463, 0.76      , 0.57575758],
       [0.73684211, 0.44444444, 0.8       , 0.57142857],
       [0.73684211, 0.44444444,

#### 2nd gen

In [25]:
params_2nd[np.argsort(eval_val_2nd[:, 3])[::-1]]

array([{'model': 'RFC', 'max_depth': 3, 'criterion': 'gini', 'min_impurity_decrease': 0.028518118316533937, 'min_samples_leaf': 0.03840696672803383},
       {'model': 'RFC', 'max_depth': 9, 'criterion': 'gini', 'min_impurity_decrease': 0.026243045130343746, 'min_samples_leaf': 13},
       {'model': 'RFC', 'max_depth': 8, 'criterion': 'gini', 'min_impurity_decrease': 0.018301587069873247, 'min_samples_leaf': 8},
       {'model': 'RFC', 'max_depth': 2, 'criterion': 'entropy', 'min_impurity_decrease': 0.025819863852279256, 'min_samples_leaf': 0.0025218655148053657},
       {'model': 'RFC', 'max_depth': 2, 'criterion': 'entropy', 'min_impurity_decrease': 0.026831972084402512, 'min_samples_leaf': 3},
       {'model': 'RFC', 'max_depth': 2, 'criterion': 'entropy', 'min_impurity_decrease': 0.014011887389787392, 'min_samples_leaf': 0.013480633167493062},
       {'model': 'RFC', 'max_depth': 7, 'criterion': 'gini', 'min_impurity_decrease': 0.007989889719185194, 'min_samples_leaf': 5},
       {'

In [26]:
models_2nd[np.argsort(eval_val_2nd[:, 3])[::-1]]

array([RandomForestClassifier(max_depth=3, min_impurity_decrease=0.028518118316533937,
                              min_samples_leaf=0.03840696672803383, random_state=42)  ,
       RandomForestClassifier(max_depth=9, min_impurity_decrease=0.026243045130343746,
                              min_samples_leaf=13, random_state=42)                   ,
       RandomForestClassifier(max_depth=8, min_impurity_decrease=0.018301587069873247,
                              min_samples_leaf=8, random_state=42)                    ,
       RandomForestClassifier(criterion='entropy', max_depth=2,
                              min_impurity_decrease=0.025819863852279256,
                              min_samples_leaf=0.0025218655148053657, random_state=42),
       RandomForestClassifier(criterion='entropy', max_depth=2,
                              min_impurity_decrease=0.026831972084402512,
                              min_samples_leaf=3, random_state=42)       ,
       RandomForestClassifier(criter

In [27]:
eval_val_2nd[np.argsort(eval_val_2nd[:, 3])[::-1]]

array([[0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.80701754, 0.54285714, 0.76      , 0.63333333],
       [0.80701754, 0.54285714, 0.76      , 0.63333333],
       [0.78947368, 0.51282051, 0.8       , 0.625     ],
       [0.78947368, 0.51282051, 0.8       , 0.625     ],
       [0.78947368, 0.51282051, 0.8       , 0.625     ],
       [0.79824561, 0.52777778, 0.76      , 0.62295082],
       [0.79824561, 0.52777778, 0.76      , 0.62295082],
       [0.80701754, 0.54545455, 0.72      , 0.62068966],
       [0.78070175, 0.5       , 0.8       , 0.61538462],
       [0.78070175, 0.5       , 0.8       , 0.61538462],
       [0.78070175, 0.5       , 0.8       , 0.61538462],
       [0.78947368, 0.51351351, 0.76      , 0.61290323],
       [0.78070175, 0.5       , 0.76      , 0.6031746 ],
       [0.78070175, 0.5       , 0.76      , 0.6031746 ],
       [0.78070175, 0.5       , 0.76      , 0.6031746 ],
       [0.78947368, 0.51428571, 0.72      , 0.6       ],
       [0.78947368, 0.51428571,

#### last gen

In [28]:
params_nth[np.argsort(eval_val_nth[:, 3])[::-1]]

array([{'model': 'RFC', 'max_depth': 10, 'criterion': 'gini', 'min_impurity_decrease': 0.02857832973001634, 'min_samples_leaf': 5},
       {'model': 'RFC', 'max_depth': 6, 'criterion': 'gini', 'min_impurity_decrease': 0.0018657030710974576, 'min_samples_leaf': 2},
       {'model': 'RFC', 'max_depth': 7, 'criterion': 'entropy', 'min_impurity_decrease': 0.0014944170828121717, 'min_samples_leaf': 0.006798413667999892},
       {'model': 'RFC', 'max_depth': 10, 'criterion': 'entropy', 'min_impurity_decrease': 0.049270080206568964, 'min_samples_leaf': 7},
       {'model': 'RFC', 'max_depth': 3, 'criterion': 'entropy', 'min_impurity_decrease': 0.05000753235240203, 'min_samples_leaf': 0.045440760793165946},
       {'model': 'RFC', 'max_depth': 3, 'criterion': 'entropy', 'min_impurity_decrease': 0.05029206078104944, 'min_samples_leaf': 0.060176024419846305},
       {'model': 'RFC', 'max_depth': 3, 'criterion': 'gini', 'min_impurity_decrease': 0.028518118316533937, 'min_samples_leaf': 0.03840696

In [29]:
models_nth[np.argsort(eval_val_nth[:, 3])[::-1]]

array([RandomForestClassifier(max_depth=10, min_impurity_decrease=0.02857832973001634,
                              min_samples_leaf=5, random_state=42)                    ,
       RandomForestClassifier(max_depth=6, min_impurity_decrease=0.0018657030710974576,
                              min_samples_leaf=2, random_state=42)                     ,
       RandomForestClassifier(criterion='entropy', max_depth=7,
                              min_impurity_decrease=0.0014944170828121717,
                              min_samples_leaf=0.006798413667999892, random_state=42),
       RandomForestClassifier(criterion='entropy', max_depth=10,
                              min_impurity_decrease=0.049270080206568964,
                              min_samples_leaf=7, random_state=42)       ,
       RandomForestClassifier(criterion='entropy', max_depth=3,
                              min_impurity_decrease=0.05000753235240203,
                              min_samples_leaf=0.045440760793165946, ra

In [30]:
eval_val_nth[np.argsort(eval_val_nth[:, 3])[::-1]]

array([[0.80701754, 0.54054054, 0.8       , 0.64516129],
       [0.83333333, 0.60714286, 0.68      , 0.64150943],
       [0.83333333, 0.60714286, 0.68      , 0.64150943],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.79824561, 0.52631579, 0.8       , 0.63492063],
       [0.80701754, 0.54285714, 0.76      , 0.63333333],
       [0.80701754, 0.54285714, 0.76      , 0.63333333],
       [0.79824561, 0.52777778, 0.76      , 0.62295082],
       [0.79824561, 0.52777778, 0.76      , 0.62295082],
       [0.78947368, 0.51351351, 0.76      , 0.61290323],
       [0.78947368, 0.51351351, 0.76      , 0.61290323],
       [0.78947368, 0.51351351, 0.76      , 0.61290323],
       [0.79824561, 0.52941176, 0.72      , 0.61016949],
       [0.78070175, 0.5       ,

### Final Evaluation on the best model

In [31]:
# model evaluation process 
# input: image infomation (dataframe), model
# output: -

def evaluate_final(df, model): 
    # training_size is ratio of training data for train-test split (float)
    # param is set of parameter for model training (dict)
    
    X_test = df.drop('label', axis=1)
    y_test = df['label']
    
    X_test = X_test.reset_index().drop('index', axis=1)
    y_test = y_test.reset_index().drop('index', axis=1)
    y_pred_prob = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    
    for i in range(len(y_pred_prob)):
        print('Actual: {} | Predicted: {} | Prob class 1: {}'.format(y_test.iloc[i]['label'], y_pred[i], y_pred_prob[i][1]))
        
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))
    print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
    print('Precision: {:.4f}'.format(precision_score(y_test, y_pred)))
    print('Recall: {:.4f}'.format(recall_score(y_test, y_pred)))
    

In [32]:
# pick the best model of the last generation
best_model = models_nth[list(np.argsort(eval_val_nth[:, 3])[::-1])][0]

In [33]:
evaluate_final(df_test, best_model)

Actual: 0 | Predicted: 0 | Prob class 1: 0.3647786726797529
Actual: 0 | Predicted: 0 | Prob class 1: 0.49550547165699155
Actual: 0 | Predicted: 1 | Prob class 1: 0.6010647378982577
Actual: 0 | Predicted: 0 | Prob class 1: 0.4673766983177701
Actual: 0 | Predicted: 0 | Prob class 1: 0.38365528642396574
Actual: 0 | Predicted: 0 | Prob class 1: 0.4525497933532968
Actual: 0 | Predicted: 0 | Prob class 1: 0.32768273558174044
Actual: 0 | Predicted: 1 | Prob class 1: 0.5520647564447259
Actual: 0 | Predicted: 0 | Prob class 1: 0.29539904142777457
Actual: 0 | Predicted: 0 | Prob class 1: 0.3124790915128204
Actual: 0 | Predicted: 0 | Prob class 1: 0.3833966168607887
Actual: 0 | Predicted: 0 | Prob class 1: 0.46350557515324486
Actual: 0 | Predicted: 0 | Prob class 1: 0.3948864355151241
Actual: 0 | Predicted: 1 | Prob class 1: 0.5319966221197678
Actual: 0 | Predicted: 0 | Prob class 1: 0.3855922418771611
Actual: 0 | Predicted: 1 | Prob class 1: 0.5450791128800404
Actual: 0 | Predicted: 0 | Prob cla

### Save model and parameter setting of model

In [34]:
# model
filename = 'checkpoints/best_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

In [35]:
# parameter setting
f = open('checkpoints/best_model.txt', 'w')
f.writelines('{}\n'.format(params_nth[(np.argsort(eval_val_nth[:, 3])[::-1])][0]))
f.close()